In [ ]:
# default_exp utils

# Utils

> General purpose utils functions

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

import os
from typing import Dict, Any


In [ ]:
#export


def dest(destination_dataset_table, prefix_dataset='tmp', return_dataset_only=False, return_table_only=False) -> Dict[str, Any]:
    """If `AIRFLOW_ENV != PROD` then write results to `prefix_dataset` instead.

    :param destination_dataset_table: destination to write results to.
    :return: destination_dataset_table: destination to write results to with prefix added if needed.
    """

    AIRFLOW_ENV = os.environ.get("AIRFLOW_ENV", "UNK")

    if AIRFLOW_ENV != 'PROD':
        destination_dataset_table_list = destination_dataset_table.replace(':', '.').split('.')
        destination_project = destination_dataset_table_list[0]
        destination_dataset = prefix_dataset
        destination_table = f'{destination_dataset_table_list[1]}_{destination_dataset_table_list[2]}'
        destination_dataset_table = f'{destination_project}.{destination_dataset}.{destination_table}'

    destination_parts = destination_dataset_table.split('.')

    if return_dataset_only == True:
        return destination_parts[1]
    elif return_table_only == True:
        return destination_parts[2]
    else:
        return destination_dataset_table


def dest_dict(destination_dataset_table, prefix_dataset='tmp') -> Dict[str, str]:
    """Wrapper for `dest()` but to return as dict.
    """
    destination_dataset_table = dest(destination_dataset_table, prefix_dataset)
    destination_parts = destination_dataset_table.split('.')
    return {
        "projectId": destination_parts[0],
        "datasetId": destination_parts[1],
        "tableId": destination_parts[2]
    }


def sched(schedule: Any) -> Any:
    """If AIRFLOW_ENV != PROD then schedule should be `@once`.

    :param schedule: schedule for prod.
    :return: schedule: `schedule` if prod else `@once`.
    """

    AIRFLOW_ENV = os.environ.get("AIRFLOW_ENV", "UNK")

    if AIRFLOW_ENV == 'PROD':
        return schedule
    else:
        return '@once'

In [ ]:
#tests
#hide
os.environ.pop('AIRFLOW_ENV', None)
assert sched('foo') == '@once'
os.environ['AIRFLOW_ENV'] = 'PROD'
assert sched('foo') == 'foo'

assert dest_dict('p.d.t', prefix_dataset='') == {'projectId': 'p', 'datasetId': 'd', 'tableId': 't'}

os.environ.pop('AIRFLOW_ENV', None)
assert dest('p.d.t') == 'p.tmp.d_t'
assert dest('p.d.t', return_dataset_only=True) == 'tmp'
assert dest('p.d.t', return_table_only=True) == 'd_t'